In [1]:
import pandas as pd
import numpy as np
DATA_DIR = "/Users/patrickkennedy/Desktop/Data_Science_MISC/Kaggle"
train = pd.read_csv(DATA_DIR + "/BNP_Paribas/train.csv")
test = pd.read_csv(DATA_DIR + "/BNP_Paribas/test.csv")
all_data = pd.concat([train, test], axis=0)
all_data = all_data.reset_index()
pd.set_option('display.max_columns', None)
foo = lambda x: pd.Series([i for i in reversed(list(str(x)))])
v22 = all_data['v22'].apply(foo)
v22.rename(columns={0:'v22-0',1:'v22-1',2:'v22-2', 3:'v22-3'},inplace=True)
v22 = v22.replace(to_replace='[a-z]', value=-1, regex=True)

v56 = all_data['v56'].apply(foo)
v56.rename(columns={0:'v56-0',1:'v56-1',2:'v56-2'},inplace=True)
v56 = v56.replace(to_replace='[a-z]', value=-1, regex=True)

v125 = all_data['v125'].apply(foo)
v125.rename(columns={0:'v125-0',1:'v125-1',2:'v125-2'},inplace=True)
v125 = v125.replace(to_replace='[a-z]', value=-1, regex=True)
data = pd.concat([all_data, v22, v56, v125], axis=1)
data = data.drop(['v22', 'v56', 'v125'], axis=1)

data.fillna(-1, inplace=True)

categorical = []

for col in data.columns:
    s = str(data[col][0])
    d = str(data[col][1])
    f = str(data[col][2])
    g = str(data[col][3])
    h = str(data[col][4])
    try:
        float(s)
        float(d)
        float(f)
        float(g)
        float(h)
        
    except ValueError:
        categorical.append(col)

for cat in categorical:
    data[cat] = pd.factorize(data[cat])[0]
    
data['mean'] = data.mean(axis=1)
data['max'] = data.max(axis=1)
data['min'] = data.min(axis=1)
data['sum'] = data.sum(axis=1)


In [ ]:
import pandas as pd
import numpy as np
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn import ensemble
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
import random
from xgboost import XGBClassifier
import xgboost as xgb
from sknn.mlp import Classifier, Layer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

#print('Load data...')
train = data[data["target"]>=0]
test = data[data["target"]<0]

cols_to_drop = ['ID', 'index', 'target']
target = train['target'].values
train = train.drop(cols_to_drop, axis=1)
id_test = test['ID'].values
test_target = test['target'].values
test = test.drop(['ID', 'index', 'target'],axis=1)

y_pred = []
holder = []

#X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.33, random_state=42)

for i in range(25):
    
    dummy = random.randint(1,10000)
    x = True
    while x == True:
        print dummy, str(len(holder)+1), holder
        #print holder
        if dummy in holder:
            dummy = random.randint(1,10000)
        else:
            x = False
    holder.append(dummy)
    
    random.seed(dummy)

    xgtrain = xgb.DMatrix(train, target)
    xgtest = xgb.DMatrix(test, label=test_target)
    
    params = {
        "objective":"binary:logistic",
        "min_child_weight":random.randint(1,100),
        "eta":float(random.randint(1,100)/1000.0),
        "max_depth":random.randint(4,12)
    }
    print params
    model = xgb.train(params, xgtrain, 250) 
    
    
    y_pred.append(model.predict(xgtest, ntree_limit=model.best_iteration))
    print y_pred[-1]
    
pd.DataFrame({"ID": id_test, "PredictedProb": np.mean(y_pred, axis=0)}).to_csv('XGBoost-native_randomparams_25iters_engineeredfeatures.csv',index=False)

5664 1 []
{'objective': 'binary:logistic', 'eta': 0.083, 'max_depth': 4, 'min_child_weight': 6}
[ 0.15199339  0.87649053  0.85952497 ...,  0.83490026  0.86928219
  0.43194804]
2181 2 [5664]
{'objective': 'binary:logistic', 'eta': 0.048, 'max_depth': 9, 'min_child_weight': 15}
[ 0.15579863  0.87168652  0.77954632 ...,  0.7980864   0.91661507
  0.44789368]
695 3 [5664, 2181]
{'objective': 'binary:logistic', 'eta': 0.073, 'max_depth': 8, 'min_child_weight': 34}

In [9]:
#what about taking the correlation of each set of predictions in the model (y_pred) before it averages?

[array([ 0.02808106,  0.95732737,  0.2404487 , ...,  0.58100545,
         0.93124896,  0.19091085], dtype=float32)]